# Customizing codebase embeddings with a projection matrix

## Goal

The outcome of this exercise is to learn a projection matrix that tailors embeddings for a codebase retrieval use case, and then measure the improvement in retrieval quality.

The notebook is mostly filled out, but has a series of small gaps that you will need to fill in (everywhere you see a "TODO" comment):
- Define the similarity functions (both basic and with projection matrix)
- Define a suitable loss function
- Construct examples for training from the pre-existing dataset
- Complete the training loop code
- Finish the retrieval function logic
- Evaluate the improvement in retrieval quality

## Background

A basic retrieval augmented generation (RAG) system will typically use embeddings to represent a set of documents that are to be searched over. Then the user input can also be converted to an embedding, and the system will use the dot product of the two embeddings to determine the relevance of the input to the documents in the database.

Many embedding models are "symmetric", which means that they treat user input text and documents (e.g. code snippets) in the same way. It might be preferable to calculate the embedding differently ("asymmetrically") for the user input because is is a fundamentally different type of text.

One way of doing this is to use the same embedding model, and then apply a matrix multiplication to the embedding of the user input. What we'll try to do here is find such a matrix that can improve retrieval quality.

## Environment

We recommend using a virtual environment to install the necessary packages.

```bash
python -m venv env
source env/bin/activate
```

### Install packages

```bash
pip install -r requirements.txt
```

## Setup

Here we generate a sample embedding with `sentence_transformers`

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import numpy as np
import torch
from sentence_transformers import SentenceTransformer

# Load a pre-trained model (this will be slow the first time)
model = SentenceTransformer("all-MiniLM-L12-v2")

def embed(text):
    embedding = model.encode([text])[0]
    return torch.tensor(embedding, dtype=torch.float32)

embedding = embed("Hello world")
dim = len(embedding)

print(f"Embedding dimension: {dim}")
print(f"Embedding: [{embedding[0]}, {embedding[1]}, ..., {embedding[-1]}]")


## Similarity

First, we'll define our definition of similarity. This can be calculated using a dot product between two embeddings. For example, if we were trying to find the similarity between a user input $x_i$ and a code snippet $x_c$, then the similarity would be

$$h(x_i, x_c) = e(x_i) \cdot e(x_c)$$

Fill out the function below:

In [ ]:
# Define the similarity function using torch
def similarity(x_i, x_c):
    # TODO
    raise NotImplementedError

# Calculate the similarity between two strings
x_i = "Where in the codebase do we do auth?"
x_c_1 = "```python\n# Authentication\ndef authenticate(username, password):\n    # Code to authenticate the user\n```"
x_c_2 = "function sum(a, b) {\n    return a + b;\n}"

similarity1 = similarity(x_i, x_c_1)
similarity2 = similarity(x_i, x_c_2)
print(f"Similarity 1: {similarity1}")
print(f"Similarity 2: {similarity2}")

## Similarity with projection matrix

Next, we'll calculate similarity using the projection matrix

$$h_\theta(x_i, x_c) = e(x_c) \theta e(x_i)$$

Fill in the function below:

In [ ]:
def similarity_with_projection(x_i, x_c, P):
    # TODO
    raise NotImplementedError

# Generate a dim by dim random matrix
P_random = torch.randn(dim, dim, dtype=torch.float32)
print(P_random)

# Calculate the similarity with the random projection matrix
similarity_with_projection1 = similarity_with_projection(x_i, x_c_1, P_random)
similarity_with_projection2 = similarity_with_projection(x_i, x_c_2, P_random)
print(f"Similarity with projection 1: {similarity_with_projection1}")
print(f"Similarity with projection 2: {similarity_with_projection2}")

## Load dataset

To train and test a matrix that is more helpful than the random one above, we will use a pre-existing dataset, which includes a list of (question, relevant code snippets) pairs, which happen to have been generated by a language model.

In [ ]:
# Load the dataset from XML file (dataset.xml)

import xml.etree.ElementTree as ET
from dataclasses import dataclass
from typing import List

@dataclass 
class Example:
    user_input: str
    snippets: List[str]

class DatasetParser:
    def __init__(self, xml_file: str):
        self.tree = ET.parse(xml_file)
        self.root = self.tree.getroot()

    def parse(self) -> List[Example]:
        examples = []
        
        for example in self.root.findall('example'):
            user_input = example.find('user_input').text
            snippets_list = []
            
            for snippet in example.find('snippets').findall('snippet'):
                # Extract code and filename from the snippet text
                snippet_text = snippet.text.strip()
                
                # Parse the filename from the code block header
                first_line = snippet_text.split('\n')[0]
                filename = first_line.split(' ')[1] if len(first_line.split(' ')) > 1 else None
                
                # Remove the code block markers and get just the code
                code_lines = snippet_text.split('\n')[1:-1]
                code = '\n'.join(code_lines)
                
                snippets_list.append(code)
                
            examples.append(Example(
                user_input=user_input,
                snippets=snippets_list
            ))
            
        return examples


parser = DatasetParser('dataset.xml')
dataset = parser.parse()


## Construct examples

Convert the dataset into a set of examples that can be used to train the projection matrix. These should include both examples of input/snippet pairs where the snippet is relevant, and pairs where the snippet is not relevant.

In [ ]:
# Next, you should generate a list of positive and negative pairs from the dataset
# These will be used to train the matrix

# TODO: Create example pairs from the dataset
example_pairs = []  # list of tuples (user input, code snippet, 1 if snippet is relevant to user input else 0)

In [ ]:
# Here we split the example pairs into training and validation sets
np.random.shuffle(example_pairs)
split_index = int(0.8 * len(example_pairs))
train_pairs = example_pairs[:split_index]
val_pairs = example_pairs[split_index:]

print(f"Number of training pairs: {len(train_pairs)}")
print(f"Number of validation pairs: {len(val_pairs)}")

## Define a loss function

With a model to calculate similarity, and a dataset of positive and negative examples, we're almost ready to train. The last thing we need is a loss function. Design a loss function that is suitable for this use case.

In [ ]:
def loss_func(predictions, targets):
    # TODO
    raise NotImplementedError

## Train the projection matrix

The entire training loop has been set up, except for a couple of lines to calculate the prediction given an example pair and to get $y$, which will then be used together to calculate the loss.

In [ ]:
import torch.optim as optim

# Initialize the projection matrix P
P = torch.randn(
    dim, dim, requires_grad=True
)

# Set hyperparameters
lr = 0.1
num_epochs = 25
optimizer = optim.Adam([P], lr=lr)
epochs, types, losses, accuracies, matrices = [], [], [], [], []

for epoch in range(num_epochs):
    # Reset gradients
    optimizer.zero_grad()

    # Iterate through training pairs
    for pair in train_pairs:
        # TODO: Get `prediction` and `y` to pass to `loss_func`
        prediction = 0 # <-- TODO
        y = 0 # <-- TODO

        loss = loss_func(prediction, y)
        loss.backward()
    
    # Update weights using Adam optimizer
    optimizer.step()

    # Calculate validation loss
    val_loss = 0
    for pair in val_pairs:
        # TODO: Get `prediction` and `y` to pass to `loss_func`
        prediction = 0 # <-- TODO
        y = 0 # <-- TODO
        
        val_loss += loss_func(prediction, y)

    print(f"Epoch {epoch}/{num_epochs}: validation loss: {val_loss.item() / len(val_pairs)}")

## Retrieval strategy

We now have a potentially improved embedding model, but need to use it for retrieval. Finish the retrieval function, which will take a user input and return relevant code snippets from the full list. Note: a vector database is not necessary.

In [ ]:
all_snippets = []

for example in dataset:
    for snippet in example['snippets']:
        all_snippets.append(snippet)

# Use similarity search with the embeddings model to retrieve relevant snippets
def retrieve_relevant_snippets(user_input: str):
    # TODO
    raise NotImplementedError

## Evaluate the new retrieval strategy

If the loss was lower by the last epoch, then we know that we improved the similarity function (at least for the validation set), but we still need a way of evaluating the retrieval strategy as a whole.

Your last task is to design an evaluation metric suitable for codebase retrieval, which we can run over the examples in the above dataset. The result of the evaluation should be a single number that attempts to represent the quality of the retrieval strategy.

In [ ]:
def evaluate_retrieval_strategy(retrieval_strategy):
    # TODO
    raise NotImplementedError

result = evaluate_retrieval_strategy(retrieve_relevant_snippets)
print(result)